In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML
from numba import njit

In [ ]:
@njit
def update_contour(candidate_loc, contour, domain):
    n = len(contour)
    x = candidate_loc[0]
    y = candidate_loc[1]
    domain[x, y] = 0
    contour[x, y] = 1
    neighbours = []
    if x > 0:
        neighbours.append([x-1, y])
    if y > 0:
        neighbours.append([x, y-1])
    if x < n-1:
        neighbours.append([x+1, y])
    if y < n-1:
        neighbours.append([x, y+1])
    for neigh in neighbours:
        if contour[neigh[0], neigh[1]] != 1 and neigh[0] in range(1, n-1) and neigh[1] in range(1, n-1):
            contour[neigh[0], neigh[1]] = 2
    candidates = [[x, y] for x in range(0,n) for y in range(0,n) if contour[x,y] == 2]
    return contour, candidates

def aggregate_candidate(candidates, domain, eta):
    a = [(np.abs(domain[i, j]))**eta for [i, j] in candidates]
    prob_cand = [c/np.sum(a) for c in a]
    agg_cand = candidates[np.random.choice(len(candidates), p = prob_cand)]
    return agg_cand

@njit
def sor_iteration(matrix, contour, w):
    n = len(matrix)
    next_matrix = np.copy(matrix)
        
    for j in range (1, n-1):
        if contour[j,0] == 1:
            next_matrix[j, 0] = 0
        else:
            # west boundary case where x = 0:
            next_matrix[j,0] = w/4 * (matrix[j,1] + matrix[j,-2] + matrix[j+1,0] + next_matrix[j-1,0]) + (1 - w) * matrix[j, 0]

        # non-boundary case 
        for i in range (1, n-1):
            if contour[j,i] == 1:
                next_matrix[j, i] = 0
            else:
                next_matrix[j,i] = w/4 * (matrix[j,i+1] + next_matrix[j,i-1] + matrix[j+1,i] + next_matrix[j-1,i]) + (1 - w) * matrix[j, i]
        if contour[j,-1] == 1:
            next_matrix[j, -1] = 0
        else:
            # east boundary case where x = n:
            next_matrix[j,-1] = w/4 * (matrix[j,1] + matrix[j,-2] + matrix[j+1,-1] + next_matrix[j-1,-1]) + (1 - w) * matrix[j, -1]
    return next_matrix

@njit
def sor(domain, contour, w, eps = 1e-5):
    conv = 1
    iter = 0
    while conv > eps:
        next_domain = sor_iteration(domain, contour, w)
        diff = np.abs(next_domain - domain)
        conv = np.max(diff)
        domain = next_domain
        iter += 1
    return next_domain, iter

def next_step(domain, contour, candidates, eta, w):
    cand = aggregate_candidate(candidates, domain, eta)
    contour, candidates = update_contour(cand, contour, domain)
    domain, _ = sor(domain, contour, w)
    return domain, contour, candidates

In [ ]:
n = 100 # grid length (once squared)
eta = 2 # shape parameter
w = 1.8
N = n**2 //40

# Initialize domain and contour
domain = np.zeros((n, n))
domain[0, :] = 1
contour = np.zeros((n, n))
contour, candidates = update_contour([n-2, n//2], contour, domain)
domain, iter = sor(domain, contour, w)

for _ in range(N):
    cand = aggregate_candidate(candidates, domain, eta)
    contour, candidates = update_contour(cand, contour, domain)
    domain, iter = sor(domain, contour, w)

color_domain = np.copy(domain)
for x in range(0, n):
    for y in range(0, n):
        if contour[x, y] == 1:
            color_domain[x, y] = -1

im = plt.imshow(color_domain, cmap='hot_r')
plt.xlabel("x (100)")
plt.ylabel("y (100)")
plt.title(fr"Diffusion Limited Aggregation with $\omega=${w} and $\eta=${eta}")
plt.xlim(0, n-1)
plt.ylim(n-1, 0)
plt.savefig("dla_set2_2a.png", dpi = 300)
plt.show()

In [ ]:
n = 100 # grid length (once squared)
eta = 2 # shape parameter
w = 1.7

# Initialize domain and contour
domain = np.zeros((n, n))
domain[0, :] = 1
contour = np.zeros((n, n))
contour, candidates = update_contour([n-2, n//2], contour, domain)
domain, iter = sor(domain, contour, w)

fig, ax = plt.subplots()
im = ax.imshow(color_domain, cmap='hot_r', animated=True)

ax.set_xlabel("x (100)")
ax.set_ylabel("y (100)")
ax.set_title(fr"Diffusion Limited Aggregation with $\omega=${w} and $\eta=${eta}")

def update(frame):
    global domain, contour, candidates
    domain, contour, candidates = next_step(domain, contour, candidates, eta, w)
    color_domain = np.copy(domain)
    for x in range(0, n):
        for y in range(0, n):
            if contour[x, y] == 1:
                color_domain[x, y] = -5
    im.set_array(color_domain)
    return [im]


ani = animation.FuncAnimation(fig, update, frames=400, interval=50, blit=False)

HTML(ani.to_jshtml())